In [4]:
!pip install google.generativeai
!pip install dotenv

  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
Using cached python_dotenv-1.1.0-py3-none-any.whl (20 kB)

   ---------------------------------------- 2/2 [dotenv]



In [ ]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

# Cargar variables de entorno desde .env
load_dotenv()

# Cargar API key desde variables de entorno
API_KEY = os.getenv("GOOGLE_GEMINI_API_KEY")
if not API_KEY:
    raise ValueError("GOOGLE_GEMINI_API_KEY no encontrada en el archivo .env")

# Configurar la API
genai.configure(api_key=API_KEY)



In [ ]:
def mejorar_etiqueta(etiqueta_original, max_intentos=3):
    # Usar el modelo gemini-1.5-flash-latest
    model = genai.GenerativeModel('gemini-1.5-flash-latest')
    
    # Crear el prompt para mejorar la etiqueta
    prompt = f"""Mejora la siguiente etiqueta para una imagen, manteniendo los elementos clave pero haciéndola más detallada y efectiva para generación de imágenes:

Etiqueta original: "{etiqueta_original}"

Instrucciones:
1. Mantén los elementos clave (pájaro blanco, bufanda roja, sombrero)
2. Añade detalles relevantes sobre el estilo, iluminación y calidad
3. Incluye términos técnicos de arte y renderizado
4. Mantén un formato similar pero más detallado
5. Asegúrate de que sea efectiva para generación de imágenes
6. Incluye términos de calidad y estilo artístico

Responde SOLO con la etiqueta mejorada, sin explicaciones adicionales."""

    for intento in range(max_intentos):
        try:
            # Generar respuesta
            response = model.generate_content(prompt)
            return response.text.strip()
        except Exception as e:
            if "429" in str(e) and intento < max_intentos - 1:
                print(f"Límite de cuota alcanzado. Esperando 30 segundos antes de reintentar...")
                time.sleep(30)  # Esperar 30 segundos antes de reintentar
                continue
            return f"Error al mejorar la etiqueta: {str(e)}"

if __name__ == "__main__":
    # Etiqueta original
    etiqueta_original = "a white bird with a red scarf and a hat on its head is standing in the air with its legs spread out, Brian Snøddy, unreal engine 5 quality render, an ambient occlusion render, photorealism"
    
    # Mejorar la etiqueta
    etiqueta_mejorada = mejorar_etiqueta(etiqueta_original)
    print("\nEtiqueta original:")
    print(etiqueta_original)
    print("\nEtiqueta mejorada:")
    print(etiqueta_mejorada)


Etiqueta original:
a white bird with a red scarf and a hat on its head is standing in the air with its legs spread out, Brian Snøddy, unreal engine 5 quality render, an ambient occlusion render, photorealism

Etiqueta mejorada:
Un pájaro blanco antropomórfico, con plumas suaves y detalladas, viste un sombrero de copa negro y una bufanda roja brillante de textura de lana. Sus patas están extendidas en una pose dinámica, suspendido en el aire contra un fondo difuminado de cielo azul crepuscular.  Iluminación volumétrica realista con rayos crepusculares, renderizado en Unreal Engine 5,  utilizando técnicas de sub-superficie scattering para las plumas,  occlusión ambiental precisa,  alto rango dinámico (HDR),  estilo fotorealista con pinceladas de arte conceptual,  detallado a nivel de película,  profundidad de campo cinemática,  8k.


In [ ]:
import os
import pandas as pd
import google.generativeai as genai
from tqdm import tqdm
import time
from dotenv import load_dotenv

# Cargar variables de entorno desde .env
load_dotenv()

# Cargar API key desde variables de entorno
API_KEY = os.getenv("GOOGLE_GEMINI_API_KEY")
if not API_KEY:
    raise ValueError("GOOGLE_GEMINI_API_KEY no encontrada en el archivo .env")

# Configurar la API
genai.configure(api_key=API_KEY)

# En notebook, el directorio actual es donde está abierto el notebook
script_dir = os.getcwd()

csv_path = os.path.join(script_dir, "outfits_with_details.csv")
tags_dir = os.path.normpath(os.path.join(script_dir, "./outfits_procesados_1024_rgb - copia/Fuzzy Bear/tags"))
output_dir = os.path.normpath(os.path.join(script_dir, "./outfits_procesados_1024_rgb - copia/Fuzzy Bear/tags_mejoradas"))

# Crear la carpeta si no existe
os.makedirs(output_dir, exist_ok=True)

print("Directorio actual:", os.getcwd())
print("Ruta del script:", script_dir)
print("Ruta absoluta del CSV:", csv_path)
print("¿Existe el archivo CSV?", os.path.exists(csv_path))
print("¿Es archivo regular?", os.path.isfile(csv_path))

def get_base_id_from_filename(filename):
    base_id = os.path.splitext(filename)[0]
    # Si tus archivos tienen siempre el patrón "_512_3c_512_3c_00" al final:
    base_id = base_id.split('_1024_3c')[0]
    return base_id

def mejorar_etiqueta_con_gemini(etiqueta_original, nombre, colaboracion, descripcion, max_intentos=3):
    """Mejora la etiqueta usando Gemini y la información del CSV"""
    model = genai.GenerativeModel('gemini-1.5-flash-latest')
    
    prompt = f"""
A partir de la siguiente información, genera una lista de etiquetas para fine-tuning de Stable Diffusion 1.5.
REQUISITOS:
- Las etiquetas deben ser concisas, separadas por comas, sin frases completas ni narrativa.
- Incluye el nombre del personaje, colaboración (si existe), atributos visuales, pose, estilo, artista, calidad, y la tipología deducida.
- Incluye siempre "fortnite character".
- Elige SOLO UNA tipología de la siguiente lista y agrégala como etiqueta, según la descripción y los ejemplos:

TIPOLOGÍAS:
- fortnite_humanoid: personaje humano con estilo clásico de Fortnite (ej: Jonesy, Midas).
- fortnite_robot_humanoid: robot con cuerpo humanoide (ej: Cromo).
- fortnite_animal_humanoid: cuerpo humano, pero la cabeza y/o algún elemento es animal (ej: Miaúsculo, Palito de Pescado).
- fortnite_fruit_humanoid: cuerpo humano, cabeza de fruta (ej: Bananín).
- fortnite_unique_humanoid: humanoides abstractos o con diseño inusual.
- fortnite_non_humanoid_animal: animal no humanoide, sus extremidades no son humanas sino del animal (ej: monturas, mascotas).
- collaboration_character: personaje de colaboración, fiel a su estilo original (ej: Goku, Peter Griffin).

- Si el fondo es negro, incluye solo "black background". Si es transparente, incluye solo "no background" o "transparent background". No incluyas ambas.
- Si hay redundancia entre etiquetas (por ejemplo, flying y airborne), elige solo la más relevante.
- Si el estilo es stylized realism, no incluyas photorealism a menos que ambos sean claramente visibles.
- Agrupa las etiquetas por: personaje, tipología, atributos, acción/pose, artista/estilo, calidad, fondo.

Nombre del personaje: {nombre}
Colaboración: {colaboracion}
Descripción: {descripcion}
Etiqueta original: "{etiqueta_original}"

Ejemplo de formato:
"flakey the snowbird, fortnite character, fortnite_animal_humanoid, white bird, red scarf, hat, detailed feathers, flying, legs spread, Brian Snøddy, unreal engine 5, ambient occlusion, cinematic lighting, stylized realism, high quality, 8k, black background"

Responde SOLO con la lista de etiquetas, separadas por comas.
"""

    for intento in range(max_intentos):
        try:
            response = model.generate_content(prompt)
            return response.text.strip()
        except Exception as e:
            if "429" in str(e) and intento < max_intentos - 1:
                print(f"Límite de cuota alcanzado. Esperando 30 segundos antes de reintentar...")
                time.sleep(30)
                continue
            return f"Error al mejorar la etiqueta: {str(e)}"

def procesar_etiquetas():
    """Procesa solo la primera etiqueta que tenga match en el CSV"""
    try:
        # Cargar el CSV
        df = pd.read_csv(csv_path)
        df['base_filename_id'] = df['id'].apply(lambda x: str(x).split('*')[0] if isinstance(x, str) else str(x))
        df.set_index('base_filename_id', inplace=True)
        print(f"CSV cargado con {len(df)} entradas.")
        
        # Obtener lista de archivos .txt
        tag_files = [f for f in os.listdir(tags_dir) if f.endswith('.txt')]
        if not tag_files:
            print(f"No se encontraron archivos .txt en '{tags_dir}'")
            return
        
        for filename in tag_files:
            base_id = get_base_id_from_filename(filename)
            tag_path = os.path.join(tags_dir, filename)
            with open(tag_path, 'r', encoding='utf-8') as f:
                etiqueta_original = f.read().strip()
            
            if base_id in df.index:
                row = df.loc[base_id]
                nombre = str(row['name']).lower() if pd.notna(row['name']) else ""
                colaboracion = str(row['builtInEmote.series.name']).lower() if pd.notna(row['builtInEmote.series.name']) else ""
                descripcion = str(row['description']).lower() if pd.notna(row['description']) else ""
                
                etiqueta_mejorada = mejorar_etiqueta_con_gemini(
                    etiqueta_original, 
                    nombre, 
                    colaboracion, 
                    descripcion
                )
                
                output_path = os.path.join(output_dir, filename)
                with open(output_path, 'w', encoding='utf-8') as f:
                    f.write(etiqueta_mejorada)
                
                print(f"\nProcesado {filename}:")
                print(f"Original: {etiqueta_original[:100]}...")
                print(f"Mejorada: {etiqueta_mejorada[:100]}...")
                print(f"\nProceso completado. 1 etiqueta procesada.")
                return  # Salir después de la primera coincidencia
        
        print("No se encontró ningún archivo con match en el CSV.")
        
    except Exception as e:
        print(f"Error durante el procesamiento: {str(e)}")

if __name__ == "__main__":
    procesar_etiquetas()

Directorio actual: c:\Users\sergi\PYTHON files\MIOTI\02-99 TFM Deep Learning
Ruta del script: c:\Users\sergi\PYTHON files\MIOTI\02-99 TFM Deep Learning
Ruta absoluta del CSV: c:\Users\sergi\PYTHON files\MIOTI\02-99 TFM Deep Learning\outfit_with_details.csv
¿Existe el archivo CSV? True
¿Es archivo regular? True
CSV cargado con 2479 entradas.

Procesado CID_069_Athena_Commando_F_PinkBear_1024_3c.txt:
Original: a pink teddy bear with a gun in its hand, cuddle team leader, royale hearts, fortnite style, fortnit...
Mejorada: cuddle team leader, fortnite character, fortnite_unique_humanoid, pink teddy bear, gun, hug pose, ro...

Proceso completado. 1 etiqueta procesada.


para ejecutar los lotes

In [ ]:
import os
import pandas as pd
import google.generativeai as genai
from tqdm import tqdm
import time
from dotenv import load_dotenv
import logging
import json
from datetime import datetime

# Cargar variables de entorno desde .env
load_dotenv()

# Cargar API key desde variables de entorno
API_KEY = os.getenv("GOOGLE_GEMINI_API_KEY")
if not API_KEY:
    raise ValueError("GOOGLE_GEMINI_API_KEY no encontrada en el archivo .env")

# Configurar la API
genai.configure(api_key=API_KEY)

# Configuración de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('etiquetas_proceso.log'),
        logging.StreamHandler()
    ]
)

# En notebook, el directorio actual es donde está abierto el notebook
script_dir = os.getcwd()

csv_path = os.path.join(script_dir, "outfits_with_details.csv")
tags_dir = os.path.normpath(os.path.join(script_dir, "../dataset/outfits_rgb_512"))
output_dir = os.path.normpath(os.path.join(script_dir, "../dataset/outfits_rgb_512/tags_mejoradas"))
progress_file = os.path.join(script_dir, "progreso_etiquetas.json")

# Asegurar que el directorio de salida existe
os.makedirs(output_dir, exist_ok=True)

def cargar_progreso():
    """Carga el progreso guardado si existe"""
    if os.path.exists(progress_file):
        try:
            with open(progress_file, 'r') as f:
                return json.load(f)
        except Exception as e:
            logging.error(f"Error al cargar el archivo de progreso: {e}")
    return {"procesados": [], "errores": {}}

def guardar_progreso(progreso):
    """Guarda el progreso actual"""
    try:
        with open(progress_file, 'w') as f:
            json.dump(progreso, f)
    except Exception as e:
        logging.error(f"Error al guardar el progreso: {e}")

def get_base_id_from_filename(filename):
    base_id = os.path.splitext(filename)[0]
    # Si tus archivos tienen siempre el patrón "_512_3c_512_3c_00" al final:
    base_id = base_id.split('_512_3c_')[0]
    return base_id

def mejorar_etiqueta_con_gemini(etiqueta_original, nombre, colaboracion, descripcion, max_intentos=3):
    """Mejora la etiqueta usando Gemini y la información del CSV"""
    model = genai.GenerativeModel('gemini-1.5-flash-latest')
    
    prompt = f"""
A partir de la siguiente información, genera una lista de etiquetas para fine-tuning de Stable Diffusion 1.5.
REQUISITOS:
- Las etiquetas deben ser concisas, separadas por comas, sin frases completas ni narrativa.
- Incluye el nombre del personaje, colaboración (si existe), atributos visuales, pose, estilo, artista, calidad, y la tipología deducida.
- Incluye siempre "fortnite character".
- Elige SOLO UNA tipología de la siguiente lista y agrégala como etiqueta, según la descripción y los ejemplos:

TIPOLOGÍAS:
- fortnite_humanoid: personaje humano con estilo clásico de Fortnite (ej: Jonesy, Midas).
- fortnite_robot_humanoid: robot con cuerpo humanoide (ej: Cromo).
- fortnite_animal_humanoid: cuerpo humano, pero la cabeza y/o algún elemento es animal (ej: Miaúsculo, Palito de Pescado).
- fortnite_fruit_humanoid: cuerpo humano, cabeza de fruta (ej: Bananín).
- fortnite_unique_humanoid: humanoides abstractos o con diseño inusual.
- fortnite_non_humanoid_animal: animal no humanoide, sus extremidades no son humanas sino del animal (ej: monturas, mascotas).
- collaboration_character: personaje de colaboración, fiel a su estilo original (ej: Goku, Peter Griffin).

- Si el fondo es negro, incluye solo "black background". Si es transparente, incluye solo "no background" o "transparent background". No incluyas ambas.
- Si hay redundancia entre etiquetas (por ejemplo, flying y airborne), elige solo la más relevante.
- Si el estilo es stylized realism, no incluyas photorealism a menos que ambos sean claramente visibles.
- Agrupa las etiquetas por: personaje, tipología, atributos, acción/pose, artista/estilo, calidad, fondo.

Nombre del personaje: {nombre}
Colaboración: {colaboracion}
Descripción: {descripcion}
Etiqueta original: "{etiqueta_original}"

Ejemplo de formato:
"flakey the snowbird, fortnite character, fortnite_animal_humanoid, white bird, red scarf, hat, detailed feathers, flying, legs spread, Brian Snøddy, unreal engine 5, ambient occlusion, cinematic lighting, stylized realism, high quality, 8k, black background"

Responde SOLO con la lista de etiquetas, separadas por comas.
"""

    for intento in range(max_intentos):
        try:
            response = model.generate_content(prompt)
            return response.text.strip()
        except Exception as e:
            if "429" in str(e) and intento < max_intentos - 1:
                print(f"Límite de cuota alcanzado. Esperando 30 segundos antes de reintentar...")
                time.sleep(30)
                continue
            return f"Error al mejorar la etiqueta: {str(e)}"

def procesar_etiquetas(reiniciar=False):
    """Procesa todas las etiquetas con manejo de errores y progreso"""
    try:
        # Cargar el progreso
        progreso = {} if reiniciar else cargar_progreso()
        archivos_procesados = set(progreso.get("procesados", []))
        errores = progreso.get("errores", {})

        # Cargar el CSV
        df = pd.read_csv(csv_path)
        df['base_filename_id'] = df['id'].apply(lambda x: str(x).split('*')[0] if isinstance(x, str) else str(x))
        df.set_index('base_filename_id', inplace=True)
        logging.info(f"CSV cargado con {len(df)} entradas.")
        
        # Obtener lista de archivos .txt
        tag_files = [f for f in os.listdir(tags_dir) if f.endswith('.txt')]
        if not tag_files:
            logging.error(f"No se encontraron archivos .txt en '{tags_dir}'")
            return

        # Filtrar archivos ya procesados
        tag_files = [f for f in tag_files if f not in archivos_procesados]
        
        for filename in tqdm(tag_files, desc="Procesando etiquetas"):
            try:
                base_id = get_base_id_from_filename(filename)
                tag_path = os.path.join(tags_dir, filename)
                
                with open(tag_path, 'r', encoding='utf-8') as f:
                    etiqueta_original = f.read().strip()
                
                if base_id in df.index:
                    row = df.loc[base_id]
                    nombre = str(row['name']).lower() if pd.notna(row['name']) else ""
                    colaboracion = str(row['builtInEmote.series.name']).lower() if pd.notna(row['builtInEmote.series.name']) else ""
                    descripcion = str(row['description']).lower() if pd.notna(row['description']) else ""
                    
                    etiqueta_mejorada = mejorar_etiqueta_con_gemini(
                        etiqueta_original, 
                        nombre, 
                        colaboracion, 
                        descripcion
                    )
                    
                    if etiqueta_mejorada.startswith("Error"):
                        raise Exception(etiqueta_mejorada)
                    
                    output_path = os.path.join(output_dir, filename)
                    with open(output_path, 'w', encoding='utf-8') as f:
                        f.write(etiqueta_mejorada)
                    
                    # Actualizar progreso
                    archivos_procesados.add(filename)
                    if filename in errores:
                        del errores[filename]
                    
                    progreso = {
                        "procesados": list(archivos_procesados),
                        "errores": errores,
                        "ultima_actualizacion": datetime.now().isoformat()
                    }
                    guardar_progreso(progreso)
                    
                    logging.info(f"Procesado exitosamente: {filename}")
                
            except Exception as e:
                error_msg = str(e)
                logging.error(f"Error procesando {filename}: {error_msg}")
                errores[filename] = {
                    "error": error_msg,
                    "timestamp": datetime.now().isoformat()
                }
                progreso["errores"] = errores
                guardar_progreso(progreso)
                continue

        # Reporte final
        total_procesados = len(archivos_procesados)
        total_errores = len(errores)
        logging.info(f"""
        Proceso completado:
        - Total de archivos procesados: {total_procesados}
        - Total de errores: {total_errores}
        - Archivos con error: {list(errores.keys())}
        """)

    except Exception as e:
        logging.error(f"Error general durante el procesamiento: {str(e)}")

if __name__ == "__main__":
    
    # Preguntar si se desea reiniciar el proceso
    reiniciar = input("¿Desea reiniciar el proceso desde cero? (s/n): ").lower() == 's'
    procesar_etiquetas(reiniciar=reiniciar)

2025-06-06 12:24:42,982 - INFO - CSV cargado con 2479 entradas.
Procesando etiquetas: 100%|██████████| 2/2 [00:01<00:00,  1.30it/s]
2025-06-06 12:24:44,528 - INFO - 
        Proceso completado:
        - Total de archivos procesados: 1485
        - Total de errores: 0
        - Archivos con error: []
        
